# Scratchpad for  I<sub>t, θ</sub> and J<sub>t, θ</sub>

[Motivation](https://terrytao.wordpress.com/2018/03/18/polymath15-sixth-thread-the-test-problem-and-beyond/)
This leaves the final range $N \leq 10$ (roughly corresponding to $x \leq 1600$). Here we can numerically evaluate $H_t(x+iy)$ to high accuracy at a fine mesh (see the data here), but to fill in the mesh we need good upper bounds on $H'_t(x+iy)$. It seems that we can get reasonable estimates using some contour shifting from the original definition of $H_t$ (see here). We are close to finishing off this remaining region and thus solving the toy problem. 

## $I_{t,\theta}$

### Preliminary math

Reference 1: [thread 2](https://terrytao.wordpress.com/2018/02/02/polymath15-second-thread-generalising-the-riemann-siegel-approximate-functional-equation/).

$H_t(z) := \int_0^\infty e^{tu^2} \Phi(u) \cos(zu)\ du,$

and $\Phi(u) := \sum_{n=1}^\infty (2\pi^2 n^4 e^{9u} - 3\pi n^2 e^{5u} ) \exp(-\pi n^2 e^{4u}),$

Shifting the contour and exchanging order of summation and integration,

$H_t(z) = \frac{1}{2} (K_{t,\theta}(z) + \overline{K_{t,\theta}(\overline{z})}) \quad (A)$

where $K_{t,\theta}(z) := \sum_{n=1}^\infty (2\pi^2 n^4 I_{t,\theta}(z-9i, \pi n^2) - 3\pi n^2I_{t,\theta}(z-5i, \pi n^2))$

with $I_{t,\theta}(b,\beta) := \int_{i\theta}^{i\theta+i\infty} \exp(tu^2 - \beta e^{4u} + ibu)\ du \quad (B)$

where $\beta = \pi n^2$,

$-\pi/8 < \theta < \pi/8$,

and $b = z-9i$ or $b = z-5i$ or, in the case of a conjugated argument $\overline{z}$ as in (A) above, $b=\overline{z}-9i$ or $b = \overline{z}-5i$. 

### Preliminary code for the series in I<sub>t, θ</sub>

#### Bounding I<sub>t, θ</sub>

Reference 2: [Bounding the derivative 2](http://michaelnielsen.org/polymath1/index.php?title=Bounding_the_derivative_of_H_t_-_second_approach)

From reference 2, equation (0),

$$I_{t,\theta}(b,\beta) = \frac{1}{2} \sum_{n=1}^\infty \left[ 2\pi^2 n^4 I_{t,\theta}(x-(9-y)i, \pi n^2) - 3\pi n^2 I_{t,\theta}(x-(5-y)i, \pi n^2) + 2\pi^2 n^4 \overline{I_{t,\theta}(x-(9+y)i, \pi n^2)} - 3\pi n^2 \overline{I_{t,\theta}(x-(5+y)i, \pi n^2)} \right]$$

where $\beta = \pi n^2$ for $n = 1, \ldots, \infty$,

and $b = x-ia$ for $a = 9+y, 9-y, 5+y, 5-y$.

Note that if the argument to $K_{t,\theta}$ is $z$ or $\overline{z}$ as in (A) above, then it has the form $x \pm iy$. Hence if $b = x \pm iy-mi$ then $b = x - i(m \mp y) = x - ia$ where $a = m \mp y$ for $m = 9, 5$. This is consistent with usage in equation (0). Thus $m = 9, 5$ should be an explicit argument for code implementing $I_{t,\theta}$.

Reference 2 points out that when $x,y > 0$, and $b=x-(9+y)i$, the $ibu$ term of the exponent in the integrand of $I_{t,\theta}$ (equation B above) will be $(9+y)u + ixu$ which would be expected to dominate cases $a=9-y$, $a=5+y$, and $a=5-y$. Considering the dominant case, the reference favors 

$\theta := \frac{\pi}{8} - \frac{1}{4} \arctan \frac{y+9}{x} \quad (1)$

which is the imaginary part of the point $u_{inflection}$ at which $\frac{d}{du} \left(-\beta e^{4u} + ibu \right)= 0$ and which should supress much of the oscillation of the integrand in (B).

In [1]:
using DBNUpperBound
using DBNUpperBound.Asymptotics
using QuadGK

θ_default(y) = π/8 - 1/4*atan((abs(y)+9)/abs(y)) # function definition

function Itθ(x, y, n, m; t=.4, upper_limit=Inf, θ = θ_default(y))
    β = π*n^2
    b = x - im*(m+y)
    return quadgk((u)->bigexp(t*u^2-β*e^(4*u)+im*b*u), 0, upper_limit) # return quadrature, error estimate
end

Itθ (generic function with 1 method)

In [2]:
@show Itθ(1600.0, .4, 10, 9); println() # integrate to infinity
@show Itθ(1600.0, .4, 10, 9, upper_limit=1.0); println() # integrate to 1
@show 1-abs(Itθ(1600.0, .4, 10, 9, upper_limit=1.0)[1]/Itθ(1600.0, .4, 10, 9)[1]); println() # compare
@show Itθ(1.0, .4, 10, 9); println()
@show Itθ(1.0, .4, 10, 9, upper_limit=1.0); println()
@show 1-abs(Itθ(1.0,.4, 10, 9, upper_limit=1.0)[1]/Itθ(1.0, .4, 10, 9)[1]); println()
nothing

Itθ(1600.0, 0.4, 10, 9) = (1.108331992409425230414830754759420402600201414092764485188646975876516212338819e-140 + 1.418332418750343368906669404277523659530081624025078068415567529503555358492418e-140im, 2.816196671428089502140567894728623301735003163566631199066810125977647265959029e-149)

Itθ(1600.0, 0.4, 10, 9, upper_limit=1.0) = (1.108331992409412029034078313944116226944510486749406350932362425023223936033518e-140 + 1.418332418750359350809905939780679734684661537870713787888600114757690580704536e-140im, 2.690824645063844088129950987256373268467070366536267755225817735427034143319125e-149)

1 - abs((Itθ(1600.0, 0.4, 10, 9, upper_limit=1.0))[1] / (Itθ(1600.0, 0.4, 10, 9))[1]) = -2.480238740835311998288664278453019007527524023368712144929829024233961108084152e-15

Itθ(1.0, 0.4, 10, 9) = (2.917554702405325988257703222828030679743217247133246721936664666619688654009133e-140 + 2.324268306210674693949968939926596238211993123498657591435697152171326943195146e-143im, 3.615838117205190255612

The quadrature accuracy is estimated as 7 to 9 decimal places, but the tail integrations from 1.0 to $\infty$ suggest the tail is 15 decimal places in magnitude below the integrals from 0 to $\infty$.

$I_{t,\theta,\geq X}(b,\beta) := \int_{i\theta+X}^{i\theta+\infty} \exp( tu^2 - \beta e^{4u} + ibu)\ du.$

When $\beta e^{4X} \cos(4\theta) \gt \max( \frac{t}{2}, \frac{a+2tX}{4}) \quad (2)$

$|I_{t,\theta,\geq X}(b,\beta)| \leq \frac{\exp( -t \theta^2 + tX^2 - \beta e^{4X} \cos(4\theta) - \theta x + aX )}{4 \beta e^{4X} \cos(4\theta) - a - 2tX}\quad (3)$

In [3]:
function XisOK(X, y, n, m; t=.4, θ=θ_default(y))
    β = π*n^2
    a = m+y
    return β*bigexp(4*X)*cos(4*θ) > max(t/2, (a+2*t*X)/4)
end

function Itθ_tail(X, x, y, n, m; t=.4, θ = θ_default(y))
    XisOK(X,y,n,m,t=t,θ=θ) || error("X is too small")
    β = π*n^2
    a = m+y
   return bigexp(-t*θ^2+t*X^2-β*e^(4*X)*cos(4*θ)-θ*x+a*X)/(4*β*e^(4*X)*cos(4*θ)-a-2*t*X)
end

Itθ_tail (generic function with 1 method)

In [4]:
@show abs(Itθ_tail(1.0, 1600.0, .4, 10, 9)/Itθ(1600.0, .4, 10, 9)[1]);println()
@show abs(Itθ_tail(1.0, 1.0, .4, 10, 9)/Itθ(1.0, .4, 10, 9)[1])
nothing

abs(Itθ_tail(1.0, 1600.0, 0.4, 10, 9) / (Itθ(1600.0, 0.4, 10, 9))[1]) = 1.861592749006765696326868260348014556019725859874554757530350171068287101326272e-7311

abs(Itθ_tail(1.0, 1.0, 0.4, 10, 9) / (Itθ(1.0, 0.4, 10, 9))[1]) = 2.775329698246869828256160952077409220652001881369772889710246261997845543776791e-7304


The provably correct bound gives much smaller values than the quadrature estimates which suggests the quadrature's limitations.

#### Bounding the tail of the infinite sum

Quoting from [the reference](https://terrytao.wordpress.com/2018/03/18/polymath15-sixth-thread-the-test-problem-and-beyond/), expression (3) "will start decaying rapidly when X is large enough (basically one needs $\beta e^{4X}cos(4\theta)$ to become large).

"For large values of n (and hence of $\beta = \pi n^2$), one can set $X=0$ and estimate the entire integral. Indeed, we conclude that if $\beta = \pi n^2$ then

$|I_{t,\theta}(b,\pi n^2)| \leq \frac{\exp( -t \theta^2 - \pi n^2 \cos(4\theta) - \theta x )}{4 \pi n^2 \cos(4\theta) - a}.$

"For instance, if $\pi n_0^2 \cos(4\theta) \gt \max(\frac{t}{2}, \frac{a}{4} )$

$|\sum_{n \geq n_0} 2\pi^2 n^4 I_{t,\theta}(x-ia, \pi n^2)| \le  \frac{2\pi^2 \exp(-t \theta^2 - \theta x - \pi n_0^2 \cos(4\theta))}{4\pi n_0^2 \cos(4\theta) - a}( \frac{n_0^4}{1 - \alpha} + 
\frac{4 n_0^3 \alpha}{(1 - \alpha)^2}  + \frac{6 n_0^2 (\alpha^2+\alpha)}{(1-\alpha)^3} + \frac{4n_0 (\alpha^3+4\alpha^2+\alpha)}{(1-\alpha)^4} + \frac{\alpha^4 + 11 \alpha^3 + 11 \alpha^2 + \alpha}{(1-\alpha)^5} ) \quad (4)$

where $\alpha := e^{-\pi n_0 \cos(4\theta)}$. Note: as rendered, $\alpha$ is easily confused with $a$ in the above expression as rendered.

Similarly,

$|\sum_{n \geq n_0} 3\pi n^2 I_{t,\theta}(x-ia, \pi n^2)|
\leq \frac{3\pi \exp(-t \theta^2 - \theta x - \pi n_0^2 \cos(4\theta))}{4\pi n_0^2 \cos(4\theta) - a} 
( \frac{n_0^2}{1-\alpha} + \frac{2n_0 \alpha}{(1-\alpha)^2} + \frac{\alpha^2+\alpha}{(1-\alpha)^3} ),$

which allows control of the tail of the series

$H_t(z) = \frac{1}{2} \sum_{n=1}^\infty 2\pi^2 n^4 I_{t,\theta}(x-(9-y)i, \pi n^2) - 3\pi n^2 I_{t,\theta}(x-(5-y)i, \pi n^2) + 2\pi^2 n^4 \overline{I_{t,\theta}(x-(9+y)i, \pi n^2)} - 3\pi n^2 \overline{I_{t,\theta}(x-(5+y)i, \pi n^2)} \quad (0)$

In [5]:
""" minimum_n_for_I(x,y,m; t= .4, θ=θ_default(y))

    Return the minimum n for which π*n^2cos(4θ)>max(t/2,a/4), where a=m+y. Note that cos(4θ)≥0.0 for allowable θ. 
    """
function minimum_n(x,y,m; t= .4, θ=θ_default(y))
    return ceil(Int, √(max(t/2, (m+y)/4)/(π*cos(4*θ))))
end

minimum_n

In [6]:
""" series_tail_Itθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9, t=t, θ=θ))

    Return a provably correct upper bound on the tail |∑2*π^2*n^4*Itθ(x,y,9)| where summation is from n0 to ∞.
    """
function series_tail_Itθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 9 # by definition of this series
    a = m+y
    return (2*π^2*bigexp(-t*θ^2-θ*x-π*n0^2*cos(4*θ)))/(4*π*n0^2*cos(4*θ)-a)*
    (n0^4/(1-α) + 4*n0^3*α/(1-α)^2 + 6*n0^2*(α^2 + α)/(1-α)^3 + 4*n0*(α^3+4*α^2+α)/(1-α)^4 + (α^4+11*α^3+11*α^2+α)/(1-α)^5)
end

series_tail_Itθ9

In [7]:
""" series_tail_Itθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5, t=t, θ=θ))

    Return a provably correct upper bound on the tail |∑3*π*n^2*Itθ(x,y,5)| where summation is from n0 to ∞.
    """
function series_tail_Itθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 5 # by definition of this series
    a = m+y
    return (3*π*bigexp(-t*θ^2-θ*x-π*n0^2*cos(4*θ)))/(4*π*n0^2*cos(4*θ)-a)*
           (n0^2/(1-α) + 2*n0*α/(1-α)^2 + (α^2 + α)/(1-α)^3)
end

series_tail_Itθ5

In [8]:
""" Ht_tail(x,y; t=.4, θ=θ_default(y), n0=max(minimum_n(x,y,5,t=t,θ=θ), minimum_n(x,y,9,t=t,θ=θ))
    
    Return an upper bound for the tail of a series expressing Ht in terms of Itθ along with the index of the tail's first term.
    """
function Ht_tail(x,y; t=.4, θ=θ_default(y), n0=max(minimum_n(x,y,5,t=t,θ=θ), minimum_n(x,y,9,t=t,θ=θ)))
    return series_tail_Itθ9(x,y,t=t,θ=θ,n0=n0)+series_tail_Itθ5(x,y,t=t,θ=θ,n0=n0) +
               series_tail_Itθ9(x,-y,t=t,θ=θ,n0=n0)+series_tail_Itθ5(x,-y,t=t,θ=θ,n0=n0), n0
end

Ht_tail

## $J_{t,\theta}$

### Preliminary math

Using reference 2:

$H'_t(z) = \frac{i}{2} \int_{i\theta}^{i\theta+\infty} e^{tu^2} \Phi(u) u e^{izu}\ du - \frac{i}{2} \overline{\int_{i\theta}^{i\theta+\infty} e^{tu^2} \Phi(u) u e^{i\overline{z}u}\ du},$

$ = \frac{i}{2} \sum_{n=1}^\infty 2\pi^2 n^4 J_{t,\theta}(x-(9-y)i, \pi n^2) - 3\pi n^2 J_{t,\theta}(x-(5-y)i, \pi n^2) - 2\pi^2 n^4 \overline{J_{t,\theta}(x-(9+y)i, \pi n^2)} + 3\pi n^2 \overline{J_{t,\theta}(x - (5+y)i, \pi n^2)} \quad (1)$

where $J_{t,\theta}(b,\beta) := \int_{i\theta}^{i\theta+\infty} \exp( tu^2 - \beta e^{4u} + ibu) u\ du.$ 

In [9]:
function Jtθ(x, y, n, m; t=.4, upper_limit=Inf, θ = θ_default(y))
    β = π*n^2
    b = x - im*(m+y)
    return quadgk((u)->im/2*bigexp(t*u^2-β*e^(4*u)+im*b*u)*u, 0, upper_limit) # return quadrature, error estimate
end

Jtθ (generic function with 1 method)

### Preliminary code for the series in $J_{t,\theta}$

Development for $J_{t,\theta}$ closely parallels that for $I_{t,\theta}$. Hence, the development below is an abbreviated parallel of the foregoing notes.

#### Bounding $J_{t,\theta}$

$J_{t,\theta,\geq X}(b,\beta) := \int_{i\theta+X}^{i\theta+\infty} \exp( tu^2 - \beta e^{4u} + ibu)u \ du.$

when, as with $I_{t,\theta}$, when X satisfies $\beta e^{4X} \cos(4\theta) \gt \max( \frac{t}{2}, \frac{a+2tX}{4})$ (equation 2 above), we have

$|J_{t,\theta,\geq X}(b,\beta)| \leq \exp( -t \theta^2 + tX^2 - \beta e^{4X} \cos(4\theta) - \theta x + aX ) (\frac{|X+i\theta|}{4 \beta e^{4X} \cos(4\theta) - a- 2tX} + \frac{1}{(4 \beta e^{4X} \cos(4\theta) - a- 2tX)^2}) \quad (6)$

In [10]:
function Jtθ_tail(X, x, y, n, m; t=.4, θ = θ_default(y))
    XisOK(X,y,n,m,t=t,θ=θ) || error("X is too small")
    β = π*n^2
    a = m+y
   return bigexp(-t*θ^2+t*X^2-β*e^(4*X)*cos(4*θ)-θ*x+a*X)*(abs(X+im*θ)/(4*β*e^(4*X)*cos(4*θ)-a-2*t*X) 
                                                           + 1/(4*β*e^(4*X)*cos(4*θ)-a-2*t*X)^2)
end

Jtθ_tail (generic function with 1 method)

#### Bounding the tail of the infinite sum

$|\sum_{n \geq n_0} 2\pi^2 n^4 J_{t,\theta}(x-ia, \pi n^2)| \leq 2\pi^2 \exp( -t \theta^2 - \pi n_0^2 \cos(4\theta) - \theta x )( \frac{\theta}{4 \pi n_0^2 \cos(4\theta) - a} + \frac{1}{(4 \pi n_0^2 \cos(4 \theta) - a)^2} )$

In [11]:
""" series_tail_Jtθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9, t=t, θ=θ))

    Return a provably correct upper bound on the tail |∑2*π^2*n^4*Jtθ(x,y,9)| where summation is from n0 to ∞.
    """
function series_tail_Jtθ9(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,9,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 9 # by definition of this series
    a = m+y
    return (2*π^2*bigexp(-t*θ^2-π*n0^2*cos(4*θ)-θ*x))*(θ/(4*π*n0^2*cos(4*θ)-a) + 1/(4*π*n0^2*cos(4*θ)-a)^2)
end

series_tail_Jtθ9

$|\sum_{n \geq n_0} 3\pi n^2 J_{t,\theta}(x-ia, \pi n^2)| \leq 3\pi \exp( -t \theta^2 - \pi n_0^2 \cos(4\theta) - \theta x )( \frac{\theta}{4 \pi n_0^2 \cos(4\theta) - a} + \frac{1}{(4 \pi n_0^2 \cos(4\theta) - a)^2} )$

In [13]:
""" series_tail_Jtθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5, t=t, θ=θ))

    Return a provably correct upper bound on the tail |∑3*π*n^2*Jtθ(x,y,5)| where summation is from n0 to ∞.
    """
function series_tail_Jtθ5(x,y; t=.4, θ=θ_default(y), n0=minimum_n(x,y,5,t=t,θ=θ))
    α = bigexp(-π*n0*cos(4*θ))
    m = 5 # by definition of this series
    a = m+y
    return 3*π*bigexp(-t*θ^2-π*n0^2*cos(4*θ)-θ*x)*(θ/(4*π*n0^2*cos(4*θ)-a) + 1/(4*π*n0^2*cos(4*θ)-a)^2)
end

series_tail_Jtθ5